## TEST

In [ ]:
### Let's visualize the embeddings ###
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


def plot_embeddings(embeddings, labels):
    pca = PCA(n_components=2)
    tsne = TSNE(n_components=2, random_state=0)
    embeddings_2d = pca.fit_transform(embeddings)
    plt.figure(figsize=(6, 4), dpi=300)
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='viridis', s=100)
    plt.title("Node Embeddings Visualization")
    plt.colorbar()
    plt.show()
plot_embeddings(node2vec.wv[train_mask_str], labels[train_mask])
# plot_embeddings(node2vec.wv[test_mask_str], labels[test_mask])